In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [4]:
df = pd.read_csv('daily-closing-prices.csv')

In [5]:
estimate_σ = lambda arr: (np.diff(arr) / arr[:-1]).std()

In [6]:
df['sigma_20'] = df.close.rolling(20).apply(estimate_σ)

C:\Users\Alex Ke\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [7]:
date_sigma = df.drop(['close'], axis=1)

In [8]:
options_df = pd.read_csv('options-df.csv').drop(['impl_volatility', 'exdate'], axis=1)

In [9]:
options_df_with_sigma = options_df.set_index('date').join(date_sigma.set_index('date'))

In [10]:
options_df_new = options_df_with_sigma.dropna(axis=0)

In [11]:
options_df_new_put = options_df_new[options_df_new.cp_flag == 'P'].drop(['cp_flag'], axis=1)

In [12]:
options_df_new_call = options_df_new[options_df_new.cp_flag == 'C'].drop(['cp_flag'], axis=1)

In [13]:
def black_scholes_put(row):
    S = row.closing_price
    X = row.strike_price / 1000
    T = row.date_ndiff / 365
    r = row.treasury_rate / 100
    σ = row.sigma_20
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    P  = norm.cdf(-d2) * X * np.exp(-r * T) - S * norm.cdf(-d1)
    return P

In [14]:
def black_scholes(row):
    S = row.closing_price
    X = row.strike_price / 1000
    T = row.date_ndiff / 365
    r = row.treasury_rate / 100
    σ = row.sigma_20
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    C = S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    return C

In [22]:
options_df_new_call.head()

,strike_price,best_bid,best_offer,volume,open_interest,date_ndiff,treasury_rate,closing_price,sigma_20
date,,,,,,,,,
19960129,600000,28.000,29.000,4,8555,47,5.17,624.22,0.007761
19960129,565000,59.250,60.250,0,9,19,5.17,624.22,0.007761
19960129,475000,152.875,153.875,0,2700,145,5.12,624.22,0.007761
19960129,570000,54.250,55.250,1,166,19,5.17,624.22,0.007761
19960129,600000,52.375,53.375,0,2155,327,5.05,624.22,0.007761


In [ ]:
options_df_new_call['black_scholes_pred'] = options_df_new_call.apply(black_scholes, axis=1)

C:\Users\Alex Ke\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


In [ ]:
options_df_new_put['black_scholes_pred'] = options_df_new_put.apply(black_scholes_put, axis=1)

In [16]:
mse = lambda df, pred_col: np.sum(np.square(df[['best_bid', 'best_offer']].mean(axis=1) - df[pred_col])) / df.shape[0]

In [145]:
# options_df_new_call['black_scholes_pred'] = options_df_new_call.apply(lambda row: black_scholes(row), axis=1)

In [18]:
mse(options_df_new_call, 'black_scholes_pred')

KeyError: 'black_scholes_pred'

In [183]:
med_abs_err = lambda df, pred_col: (np.abs(df[['best_bid', 'best_offer']].mean(axis=1) - df[pred_col])).iloc[df.shape[0]//2]

In [184]:
med_abs_err(options_df_new_call, 'black_scholes_pred')

24.75583508250861

In [185]:
options_df_new_call.shape

(6131354, 10)

In [187]:
options_df_new_call.to_csv('data/call-options-black-scholes.csv')

In [11]:
options_df_new_put['black_scholes_pred'] = options_df_new_put.apply(lambda row: black_scholes_put(row), axis=1)

/Users/ycm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


In [15]:
options_df_new_put.tail()

,strike_price,best_bid,best_offer,volume,open_interest,date_ndiff,treasury_rate,closing_price,sigma_20,black_scholes_pred
date,,,,,,,,,,
20171229,2950000,282.3,285.0,0,0,367,1.76,2673.61,0.003498,224.644711
20171229,3000000,321.8,324.7,0,22,367,1.76,2673.61,0.003498,273.767672
20171229,3050000,364.7,367.6,0,0,367,1.76,2673.61,0.003498,322.890633
20171229,3100000,399.3,419.6,0,0,367,1.76,2673.61,0.003498,372.013595
20171229,3200000,493.8,513.5,0,0,367,1.76,2673.61,0.003498,470.259517


In [31]:
bid_ask_avg = np.mean([options_df_new_put.best_bid, options_df_new_put.best_offer],axis=0)
# np.abs(bid_ask_avg - options_df_new_put.black_scholes_pred)

In [32]:
bid_ask_avg

array([  5.5625,  10.9375,   9.75  , ..., 366.15  , 409.45  , 503.65  ])

In [33]:
pred = options_df_new_put.black_scholes_pred

In [35]:
'rmse', np.sqrt(np.mean(np.square(bid_ask_avg-pred)))

('rmse', 35.06335875243403)

In [36]:
'med-err', np.median(bid_ask_avg-pred)

('med-err', 4.4375)

In [37]:
'avg-abs-err', np.mean(np.abs(bid_ask_avg-pred))

('avg-abs-err', 17.08568020404199)

In [38]:
'med abs dev', np.median(np.abs(bid_ask_avg-pred))

('med abs dev', 4.449999999999999)

In [39]:
diff = (bid_ask_avg - pred) / bid_ask_avg

In [43]:
'percents'

'rmse percent', np.sqrt(np.mean(np.square(diff))) * 100

('rmse percent', 85.65289289875021)

In [44]:
'med err percent', np.median(diff) * 100

('med err percent', 100.0)

In [46]:
'average abs err percent', np.mean(np.abs(diff))*100

('average abs err percent', 76.0467406593385)

In [47]:
'med abs err percent', np.median(np.abs(diff))*100

('med abs err percent', 100.0)